In [1]:
import pandas as pd
import pickle

In [2]:
linked_node_dic_df = pd.read_csv('linked_node_dic_df.csv', sep='\t')
# dic指明实体有多少相邻节点
entity_link_count = linked_node_dic_df['original_entity'].value_counts().to_dict()
# 相邻节点字典
linke_dic = linked_node_dic_df.groupby('original_entity')['linked_entities'].agg(list).to_dict()

In [3]:
linke_dic

{'Q1000178': ['Q30', 'Q2086913', 'Q1000178'],
 'Q1001244': ['Q30', 'Q1001244'],
 'Q1001316': ['Q484247', 'Q30', 'Q1001316'],
 'Q1002140': ['Q940748', 'Q3224', 'Q408', 'Q3957', 'Q1002140'],
 'Q1002177': ['Q145', 'Q3957', 'Q1002177'],
 'Q1002812': ['Q145', 'Q1002812'],
 'Q1002813': ['Q81965', 'Q3231690', 'Q1013717', 'Q27415', 'Q1002813'],
 'Q1002908': ['Q30', 'Q49', 'Q3134980', 'Q1002908'],
 'Q1003': ['Q36', 'Q1003'],
 'Q1003711': ['Q502294', 'Q30', 'Q1003711'],
 'Q1003761': ['Q145', 'Q1003761'],
 'Q1003780': ['Q30', 'Q476068', 'Q9640', 'Q1003780'],
 'Q100391': ['Q38', 'Q747074', 'Q100391'],
 'Q1004085': ['Q145', 'Q1004085'],
 'Q1004712': ['Q502459',
  'Q30',
  'Q1003082',
  'Q1086117',
  'Q1089095',
  'Q138311',
  'Q204151',
  'Q18432',
  'Q941023',
  'Q1004712'],
 'Q1006865': ['Q502294', 'Q30', 'Q1006865'],
 'Q1007870': ['Q838948', 'Q1007870'],
 'Q1007904': ['Q495873', 'Q30', 'Q2086913', 'Q1007904'],
 'Q1008520': ['Q30', 'Q941023', 'Q1008520'],
 'Q101017': ['Q101017'],
 'Q1010236': ['Q

In [4]:
with open('./entity_in_emb_file.pickle', 'rb') as file:
    entity_in_emb_file = pickle.load(file)

In [5]:
# 计算 训练集各新闻包含 entity数
with open('../MIND-200k/train/news.tsv', "r", encoding="utf-8") as file:
    lines = file.readlines()
data = []
for line in lines:
    parts = line.strip().split('\t')
    news_id = parts[0]
    title_entity = parts[-2]
    abstract_entity = parts[-1]
    # 获得新闻中的实体
    title_entity_id = [item["WikidataId"] for item in eval(title_entity) ]
    abstract_entity_id = [item["WikidataId"] for item in eval(abstract_entity) ]
    news_entity_id  = title_entity_id + abstract_entity_id
    news_entity_id_count = len(news_entity_id)

    link_entity = set(news_entity_id)
    # source entity也不一定有emb，也要筛选一下
    for source_entity in news_entity_id:
        if source_entity in linke_dic:
            link_entity = link_entity.union(linke_dic[source_entity])
    # 确保有emb
    link_entity = link_entity.intersection(entity_in_emb_file)
    link_entity = list(link_entity)
    link_entity_count = len(link_entity)

    data.append([news_id, news_entity_id, news_entity_id_count, link_entity, link_entity_count])
    
linked_relation_in_train_news = pd.DataFrame(data, columns=["news_id", "news_entity_id", "news_entity_id_count", 'link_entity_id', 'link_entity_count'])

In [6]:
linked_relation_in_train_news

,news_id,news_entity_id,news_entity_id_count,link_entity_id,link_entity_count
0,N88753,"[Q80976, Q43274, Q9682]",3,"[Q153330, Q151754, Q80976, Q34433, Q154920, Q4...",16
1,N23144,"[Q193583, Q193583]",2,[Q193583],1
2,N86255,[Q622899],1,"[Q622899, Q107126, Q30, Q9588, Q1553390]",5
3,N93187,[Q212],1,[Q212],1
4,N75236,[Q155223],1,[Q155223],1
...,...,...,...,...,...
75770,N114875,"[Q192081, Q20551945, Q30229860, Q371775, Q1928...",7,"[Q1317534, Q1874, Q203008, Q41466, Q207980, Q8...",23
75771,N115249,[],0,[],0
75772,N64337,"[Q39709, Q39709]",2,[Q39709],1
75773,N100102,"[Q577698, Q632511]",2,"[Q632511, Q4685084, Q577698, Q4965230, Q476028...",12


In [7]:
# 计算 dev集各新闻包含 entity数
with open('../MIND-200k/dev/news.tsv', "r", encoding="utf-8") as file:
    lines = file.readlines()
data = []
for line in lines:
    parts = line.strip().split('\t')
    news_id = parts[0]
    title_entity = parts[-2]
    abstract_entity = parts[-1]
    # 获得新闻中的实体
    title_entity_id = [item["WikidataId"] for item in eval(title_entity) ]
    abstract_entity_id = [item["WikidataId"] for item in eval(abstract_entity) ]
    news_entity_id  = title_entity_id + abstract_entity_id
    news_entity_id_count = len(news_entity_id)

    link_entity = set(news_entity_id)
    for source_entity in news_entity_id:
        if source_entity in linke_dic:
            link_entity = link_entity.union(linke_dic[source_entity])
    # 确保有emb
    link_entity = link_entity.intersection(entity_in_emb_file)        
    link_entity = list(link_entity)
    link_entity_count = len(link_entity)

    data.append([news_id, news_entity_id, news_entity_id_count, link_entity, link_entity_count])
    
linked_relation_in_dev_news = pd.DataFrame(data, columns=["news_id", "news_entity_id", "news_entity_id_count", 'link_entity_id', 'link_entity_count'])

In [8]:
linked_relation_in_dev_news

,news_id,news_entity_id,news_entity_id_count,link_entity_id,link_entity_count
0,N23144,"[Q193583, Q193583]",2,[Q193583],1
1,N86255,[Q622899],1,"[Q622899, Q107126, Q30, Q9588, Q1553390]",5
2,N93187,[Q212],1,[Q212],1
3,N75236,[Q155223],1,[Q155223],1
4,N99744,"[Q3179593, Q3179593, Q171171, Q371820]",4,"[Q60, Q30, Q3179593, Q171171, Q371820, Q1860]",6
...,...,...,...,...,...
40398,N36266,"[Q633, Q633, Q15133865]",3,"[Q633, Q15133865, Q846570]",3
40399,N31741,[Q327496],1,[Q327496],1
40400,N71674,"[Q51552, Q51552, Q7777510, Q142348]",4,"[Q36, Q142, Q11635, Q142348, Q90, Q102427, Q18...",11
40401,N124635,"[Q41148, Q515161, Q642878]",3,"[Q5901134, Q7414, Q23781155, Q5887360, Q226658...",17


In [9]:
# 计算 dev集各新闻包含 entity数
with open('../MIND-200k/test/news.tsv', "r", encoding="utf-8") as file:
    lines = file.readlines()
data = []
for line in lines:
    parts = line.strip().split('\t')
    news_id = parts[0]
    title_entity = parts[-2]
    abstract_entity = parts[-1]
    # 获得新闻中的实体
    title_entity_id = [item["WikidataId"] for item in eval(title_entity) ]
    abstract_entity_id = [item["WikidataId"] for item in eval(abstract_entity) ]
    news_entity_id  = title_entity_id + abstract_entity_id
    news_entity_id_count = len(news_entity_id)

    link_entity = set(news_entity_id)
    for source_entity in news_entity_id:
        if source_entity in linke_dic:
            link_entity = link_entity.union(linke_dic[source_entity])
    # 确保有emb
    link_entity = link_entity.intersection(entity_in_emb_file)    
    link_entity = list(link_entity)
    link_entity_count = len(link_entity)

    data.append([news_id, news_entity_id, news_entity_id_count, link_entity, link_entity_count])
    
linked_relation_in_test_news = pd.DataFrame(data, columns=["news_id", "news_entity_id", "news_entity_id_count", 'link_entity_id', 'link_entity_count'])

In [10]:
linked_relation_in_test_news

,news_id,news_entity_id,news_entity_id_count,link_entity_id,link_entity_count
0,N93187,[Q212],1,[Q212],1
1,N75236,[Q155223],1,[Q155223],1
2,N99744,"[Q3179593, Q3179593, Q171171, Q371820]",4,"[Q60, Q30, Q3179593, Q171171, Q371820, Q1860]",6
3,N124534,"[Q1215884, Q1215884]",2,"[Q41323, Q5870004, Q191615, Q319007, Q1215884,...",7
4,N51947,"[Q1406, Q94]",2,"[Q94, Q1406]",2
...,...,...,...,...,...
40777,N71674,"[Q51552, Q51552, Q7777510, Q142348]",4,"[Q36, Q142, Q11635, Q142348, Q90, Q102427, Q18...",11
40778,N124635,"[Q41148, Q515161, Q642878]",3,"[Q5901134, Q7414, Q23781155, Q5887360, Q226658...",17
40779,N99177,"[Q3304418, Q10479, Q1770490]",3,"[Q10479, Q3304418, Q1770490, Q145, Q27631]",5
40780,N97143,"[Q169946, Q5111202]",2,"[Q169946, Q5111202]",2


In [11]:
result_df = pd.concat([linked_relation_in_train_news, linked_relation_in_dev_news, linked_relation_in_test_news], ignore_index=True)

In [12]:
result_df = result_df.drop_duplicates(subset='news_id', keep='first')

In [13]:
result_df

,news_id,news_entity_id,news_entity_id_count,link_entity_id,link_entity_count
0,N88753,"[Q80976, Q43274, Q9682]",3,"[Q153330, Q151754, Q80976, Q34433, Q154920, Q4...",16
1,N23144,"[Q193583, Q193583]",2,[Q193583],1
2,N86255,[Q622899],1,"[Q622899, Q107126, Q30, Q9588, Q1553390]",5
3,N93187,[Q212],1,[Q212],1
4,N75236,[Q155223],1,[Q155223],1
...,...,...,...,...,...
155923,N130200,[],0,[],0
156242,N11927,[],0,[],0
156500,N127497,"[Q2492612, Q94689, Q94689]",3,"[Q94689, Q2492612, Q30, Q34442, Q1397, Q4955209]",6
156703,N1106,[Q2954009],1,"[Q145, Q1079023, Q2954009]",3


In [14]:
link_entuty_dic = result_df.set_index('news_id')['link_entity_id'].to_dict()

In [15]:
link_entuty_dic

{'N88753': ['Q153330',
  'Q151754',
  'Q80976',
  'Q34433',
  'Q154920',
  'Q42049',
  'Q42182',
  'Q9682',
  'Q1255921',
  'Q43274',
  'Q1860',
  'Q362',
  'Q145',
  'Q42646',
  'Q645968',
  'Q222595'],
 'N23144': ['Q193583'],
 'N86255': ['Q622899', 'Q107126', 'Q30', 'Q9588', 'Q1553390'],
 'N93187': ['Q212'],
 'N75236': ['Q155223'],
 'N99744': ['Q60', 'Q30', 'Q3179593', 'Q171171', 'Q371820', 'Q1860'],
 'N124534': ['Q41323',
  'Q5870004',
  'Q191615',
  'Q319007',
  'Q1215884',
  'Q30',
  'Q276530'],
 'N51947': ['Q94', 'Q1406'],
 'N59220': ['Q49233'],
 'N17957': [],
 'N40259': ['Q298', 'Q2887'],
 'N42222': ['Q1323662'],
 'N46520': ['Q4200953',
  'Q178600',
  'Q19809',
  'Q106010',
  'Q1365868',
  'Q3353942'],
 'N40599': [],
 'N13152': ['Q7957644', 'Q3191485', 'Q30', 'Q816', 'Q16556'],
 'N22273': [],
 'N30547': ['Q173476',
  'Q27564',
  'Q3231690',
  'Q30',
  'Q2335420',
  'Q215601',
  'Q1211586'],
 'N42639': [],
 'N54460': ['Q3403879'],
 'N79856': ['Q209330'],
 'N72751': ['Q1166'],
 'N

In [16]:
with open('link_entuty_dic.pkl', 'wb') as file:
    pickle.dump(link_entuty_dic, file)

In [ ]:
with open('link_entuty_dic.pkl', 'rb') as file:
    my_dict = pickle.load(file)

In [ ]:
my_dict

In [17]:
with open('./link_entity_dic.pkl', 'rb') as file:
    link_entity_dic = pickle.load(file)

In [20]:
link_entity_dic

{'N88753': ['Q153330',
  'Q151754',
  'Q80976',
  'Q34433',
  'Q154920',
  'Q42049',
  'Q42182',
  'Q9682',
  'Q1255921',
  'Q43274',
  'Q1860',
  'Q362',
  'Q145',
  'Q42646',
  'Q645968',
  'Q222595'],
 'N23144': ['Q193583'],
 'N86255': ['Q622899', 'Q107126', 'Q30', 'Q9588', 'Q1553390'],
 'N93187': ['Q212'],
 'N75236': ['Q155223'],
 'N99744': ['Q60', 'Q30', 'Q3179593', 'Q171171', 'Q371820', 'Q1860'],
 'N124534': ['Q41323',
  'Q5870004',
  'Q191615',
  'Q319007',
  'Q1215884',
  'Q30',
  'Q276530'],
 'N51947': ['Q94', 'Q1406'],
 'N59220': ['Q49233'],
 'N17957': [],
 'N40259': ['Q298', 'Q2887'],
 'N42222': ['Q1323662'],
 'N46520': ['Q4200953',
  'Q178600',
  'Q19809',
  'Q106010',
  'Q1365868',
  'Q3353942'],
 'N40599': [],
 'N13152': ['Q7957644', 'Q3191485', 'Q30', 'Q816', 'Q16556'],
 'N22273': [],
 'N30547': ['Q173476',
  'Q27564',
  'Q3231690',
  'Q30',
  'Q2335420',
  'Q215601',
  'Q1211586'],
 'N42639': [],
 'N54460': ['Q3403879'],
 'N79856': ['Q209330'],
 'N72751': ['Q1166'],
 'N

In [19]:
len(link_entity_dic)

78442